In [18]:
using DynamicPolynomials, SparseArrays
println("***Problem setting***")

n=3
println("Number of variables: n=",n)

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

function generate_sparse_random_poly(v)
    c=sprand(Float64,length(v),0.01)
    I,V=findnz(c)
    V=2*V.-1
    return sparsevec(I,V,length(v))'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:2))
f=generate_random_poly(v)


# unit sphere constraint
m=ceil(Int64, n/5)
g=Vector{Polynomial{true,Float64}}(undef,m)
 #type of coefficients of each polynomial must be float

randx=rand(n)# create a feasible solution
randx=rand(1)[1]*randx./sum(randx)
g[1]=1.0-sum(x)
for j in 2:m
    g[j]=generate_random_poly(v[2:end])
    g[j]+=0.125-g[j](x => randx) #make constraints feasible
end



m=length(g)
println("Number of inequality constraints: m=",m)

l=ceil(Int64,n/5)

h=Vector{Polynomial{true,Float64}}(undef,l)
for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

#h=[h;1.0-sum(x.^2)]

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variables: n=3
Number of inequality constraints: m=1
Number of equality constraints: l=1


In [19]:
include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=false);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

k=3
s=1

@time opt_val1=InterRelax.RelaxDense_without_multiplier(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek");

**Interrupted relaxation based on Handelman's Positivstellensatz**
Relaxation order: k=3
Sparsity order: s=1
Maximal matrix size:1
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : LO (linear optimization problem)
  Constraints            : 20              
  Cones                  : 0               
  Scalar variables       : 35              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 4
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number         

In [20]:
using SparseArrays
using TSSOS

k=3
pop=[[f(x=>x.^2)];[g[j](x=>x.^2) for j in 1:m];[h[j](x=>x.^2) for j in 1:l]]
@time opt,sol,data=TSSOS.cs_tssos_first(pop,x,k,numeq=l,CS=false,TS="block",solver="Mosek");

************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[3]
[1]
------------------------------------------------------
Starting to compute the block structure...
Obtained the block structure in 0.000244535 seconds. The maximal size of blocks is 4.
Assembling the SDP...
There are 26 affine constraints.
SDP assembling time: 0.000161769 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 26              
  Cones                  : 0               
  Scalar variables       : 15              
  Matrix variables       : 6               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in e

In [21]:
opt,sol,data = TSSOS.cs_tssos_higher!(data, TS="block");

Starting to compute the block structure...
No higher CS-TSSOS hierarchy!


In [22]:
include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=false);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

k=0
s=5

@time opt_val1,opt_sol=InterRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek",comp_opt_sol=false);

**Interrupted relaxation based on Putinar-Vasilescu's Positivstellensatz**
Relaxation order: k=0
Sparsity order: s=5
Maximal matrix size:4
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 10              
  Cones                  : 0               
  Scalar variables       : 12              
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - numbe

In [23]:
include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=false);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

k=1
s=31

@time opt_val1,opt_sol=InterRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek",comp_opt_sol=true);

**Interrupted relaxation based on Putinar-Vasilescu's Positivstellensatz**
Relaxation order: k=1
Sparsity order: s=31
Maximal matrix size:4
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 20              
  Cones                  : 0               
  Scalar variables       : 15              
  Matrix variables       : 5               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - numb

In [24]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

using TSSOS

k=2
vars,pop=InterRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)
@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek");

************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[3]
[1]
------------------------------------------------------
Assembling the SDP...
There are 35 affine constraints.
SDP assembling time: 0.000173948 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 35              
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 5               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - 

(-2.4539575528275797, nothing, TSSOS.mcpop_data(3, 0, 5, 1, Array{Array{UInt16,1},1}[[[], [0x0001], [0x0001, 0x0001], [0x0001, 0x0002], [0x0001, 0x0003], [0x0002], [0x0002, 0x0002], [0x0002, 0x0003], [0x0003], [0x0003, 0x0003]], [[0x0001], [0x0002], [0x0003], []], [[0x0001]], [[0x0002]], [[0x0003]], [[0x0001, 0x0001], [0x0001, 0x0002], [0x0001, 0x0003], [0x0002, 0x0002], [0x0002, 0x0003], [0x0003, 0x0003], [0x0001], [0x0002], [0x0003], []]], Array{Float64,1}[[-0.8379409669514253, -0.5376089985264545, 0.9179132440221776, -0.3239281977779882, -0.7110938606999242, -0.7262965963290466, -0.6958827823391958, -0.927722409034589, -0.9339932896528902, -0.8735419584391555], [-1.0, -1.0, -1.0, 1.0], [1.0], [1.0], [1.0], [0.7158872891598889, -0.7105161418383896, -0.16744554378135357, 0.013708205510450266, -0.36178572793631814, -0.6373327527778683, 0.14299384212621957, 0.41573308550719856, 0.5206159976277438, -0.04523600753695502]], Array{Array{Array{UInt16,1},1},1}[[[[], [0x0001], [0x0002], [0x000

In [25]:
# using SparseArrays

# include("../src/InterRelax.jl")
# using .InterRelax

# n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);

# InterRelax.save_info_densePOP(randx,n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh)